# Local Outlier Factor sur clusters
---
_Tentative d'affinement de cluster réalisé avec des applications de Kmean et SOM sur un dataset avec LOF_

---

  1. Procédure automatisé
  2. Exploration
---

### 1. Procédure automatisée
- importation des libraries nécessaires numpy, gensim, sklearn, ...

In [1]:
import random
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from os import listdir, path, makedirs
import re

- fonction d'évaluation pour évaluer rapidement chaque cluster:

In [2]:

def loadDataset(file, dico):
    """Charge un fichier keywords et le transforme en dictionnaire

    Args:
        file (fichier): fichier keywords correspondant
        dico (dictionnaire): dictionnaire vide
    """
    with open(file, 'r') as file:
        for line in file:
            resultat[line.split("\n")[0]] = 0

def recordic(file, dico):
    """Parcours un dataset et vérifie le nombre d'occurence des clefs d'un dictionnaire 

    Args:
        file (fichier): dataset
        dico (dictionnaire): dictionnaire de mot clés
    """
    dico["LINE"] = 0
    dico["OCCUR"] = 0
    with open(file, 'r') as file:
        for e in file:
            dico["LINE"] += 1
            for k in dico.keys():
                if len(k.split(" ")) == 1:
                    for word in e.split(" "):
                        word = re.sub('[^a-z0-9àâäéèëêïîôöùûüÿç]+', '', word.lower())
                        if k == word:
                            dico[k] +=1
                            dico["OCCUR"]  +=1
                else:
                    if(k in e):
                        dico[k] +=1
                        dico["OCCUR"] +=1

def rapportR(dico, out):
    """représente le rapport sur un dataset en fonction des mots clés relatif ou constitutif de sa formation

    Args:
        dico (dictionnaire): dictionnaire des mots clés
        out (fichier): fichier de sortie
    """
    with open(out, 'a') as file:
        for k in dico.keys():
            if(k != "LINE") and (dico['LINE']>0):
                file.write(f"{k}:{dico[k]}/{round(dico[k]/dico['LINE']*100, 4)}%\n")
        if dico['LINE']>0:
            file.write(f"Total occurences:{dico['OCCUR']}/{round(dico['OCCUR']/dico['LINE']*100, 4)}%\n")


- fonction d'encodage

In [3]:
def encode(msg, model, dim):
    """
        Encode un message:
    Args:
        msg (string): une chaîne de caractère correspondant aux données à encodées
        model (array): modéle d'encodage
        dim: nombre de dimensions nécessaires

    Returns:
        array d'int: vecteur de données encodées
    
    """
    return np.mean([model[word] for word in msg if word in model] or [np.zeros(dim)], axis = 0)

- fonction de sauvegarde

In [4]:
#création de dossier
def SaveLofSelect(file, index, datas):
    with open(file, 'a') as out:
        for i,e in enumerate(index):
            if e<0:
                out.write(datas[i])

- Ensemble d'élément pour le parcours:

In [5]:
paths = ["1M_Lem_clusters/", "1M_Min_clusters/", "1M_MinStp_clusters/", "1M_Std_clusters/"]
version = ["km_25/", "km_50/", "km_100/", "som_100/", "som_225/", "som_400/"]
lof = ["lofk2", "lofk3", "lofk4"]

In [6]:
for i in range(len(paths)):
    for j in range(len(version)):
        CurrentPath = paths[i]+version[j]
        clean = []
        raw = []
        #récupération de la liste des fichier clean & raw:
        for e in listdir(CurrentPath):
            if e.endswith("clean_msg.txt"):
                clean.append(e)
            elif e.endswith("raw_msg.txt"):
                raw.append(e)
            else:
                pass
        #traitement pour chaque cluster:
        for k in range(len(clean)):
            corpus = CurrentPath+clean[k]
            corpus_raw = CurrentPath+raw[k]
            model = Word2Vec(corpus_file=corpus, vector_size=300, window=5, min_count=2, workers=8, sg=1)
            wv = model.wv
            wv.save("w2vec_model_temp")
            model = KeyedVectors.load("w2vec_model_temp")
            #chargement des données en mémoire:
            FormatDataset = []
            RawData = []
            with open(corpus, "r") as file:
                lines = file.readlines()
    
            FormatDataset = [line.split() for line in lines]

            with open(corpus_raw, "r") as file:
                lines = file.readlines()
    
            RawData = [line for line in lines]
            #encore les données:
            dataset_encoded = np.array([encode(msg, model, 300) for msg in FormatDataset])
            #reduction de dimensions:
            pca = PCA(n_components=2)
            model2d = pca.fit_transform(dataset_encoded)
            #application en fonction du kNeighbots choisi
            for l in range(len(lof)):
                clf = LocalOutlierFactor(n_neighbors= l+2)
                out = clf.fit_predict(model2d)
                path_test = CurrentPath+lof[l]+"/" 
                if not path.isdir(path_test):
                    makedirs(path_test)
                outlier = CurrentPath+lof[l]+"/"+clean[k][:-4]+"_"+lof[l]+".txt"
                SaveLofSelect(outlier, out, RawData)
                resultat = {}
                loadDataset("keywords",resultat)
                recordic(outlier ,resultat)
                rapportR(resultat, outlier[:-4]+"_rapport.txt")

---

### 2. Exploration

- Meilleurs résultats par dataset et procédures:

---

_Conclusion des cluster sur un dataset lemmatisé:
    On Constate que même si on arrive à avoir des clusters avec une représentation plus élevée de vocabulaire pouvant constituer des messages haineux le nombre reste moindre. En parcourant les clusters, on remarque à défaut d'avoir une démarcation claire sur les éléments recherchés, qu'il y a bien souvent des "thématiques" qui s'articulent autour d'une suite de caractère spécifique, un verbe ou une expression._
    
---

---

_Remarque:
 Une nouvelle fois les résultats sont décevant, mais si on observe le cluster 289 de som 400 on constate que les outliers locaux contenant "fils de" sont bien sélectionnés parmi un cluster de fil, fille, filles, fils etc._ 
 
---

---